Cartar de posible desvio para la union

In [2]:
import sys
sys.path.append('..')

import pandas as pd
import collections
import os

from docxtpl import DocxTemplate

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [3]:
RUTA_COMPLETA

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
path_xlsx_avance = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [5]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='DETALLE ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE')

C:\Users\Usuario\anaconda3\envs\utea\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Usuario\anaconda3\envs\utea\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [6]:
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas['cod_cos'] = xlsx_entregas['cod_cos'].astype(int)
xlsx_entregas['cod_ca'] = xlsx_entregas['cod_ca'].astype(int)
xlsx_entregas['%ENTREGA'] = xlsx_entregas['%ENTREGA'] * 100
xlsx_entregas.head(3)

,inst,cod_cos,cod_ca,nom_ca,TOTAL COMPROMISO,TOTAL ENTREGAS,%ENTREGA
0,1,49,660,AGUILERA ANTELO ALEX ENRIQUE,982.61,627.21,63.831021
1,1,54,3361,ENRIQUEZ ROJAS ALBERTO,1615.39,1460.25,90.396127
2,1,55,3460,ESPINOZA MENDOZA SANTIAGO,5288.56,4960.50,93.796799


In [7]:
xlsx_avance.head(3)

,COD COS,COD CA,NOMBRE CAÑERO,COMPROMISO,ENTREGAS,%ENTREGA,AREA,COSECHADO,SIN COSECHAR,SEMILLA,...,PRODUCCION EST,EST TN COSECHA,DIF EST ENTREGA,%DIF EST ENTREGA,TN EN CAMPO,COMPROMISO REST,DIFERENICA,COMPRADOR,DESVIADOR,EST VS COMPRIM
0,523,42225,PIZARRO BRAVO ROLY ANDRES,1330.71,187.93,0.141225,21.844502,18.187097,3.657405,NaN,...,1744.673280,1464.826945,-1276.896945,7.794535,279.846335,1142.78,-862.933665,0.0,1.0,1.311085
1,414,41973,RIBERA JUSTINIANO JEFFER PAUL,9549.85,3140.72,0.328876,196.846310,147.152193,49.694117,NaN,...,9340.156951,7071.481942,-3930.761942,2.251548,2268.675009,6409.13,-4140.454991,0.0,1.0,0.978042
2,374,4638,LOPEZ JUANA GODOY DE,15693.28,7899.80,0.503387,301.708136,217.821435,83.886701,NaN,...,14649.901853,10725.763407,-2825.963407,1.357726,3924.138446,7793.48,-3869.341554,0.0,1.0,0.933514


In [8]:
fecha_inf = '25 de septiembre de 2025'
fecha_form = '25-09-2025'

fecha_img = ' de septiembre de 2025'

cod_grupos = [166
]


In [9]:
def crear_reporte(lista_cod_cos):
    for cod in lista_cod_cos:
        grupo = xlsx_entregas[xlsx_entregas['cod_cos'] == cod].copy()
        
        # convertir datos del grupo a objetos
        list_obj_caneros = []
        for index, row in grupo.iterrows():
            canero = {
            'inst' : row['inst'],
            'cod_ca' : row['cod_ca'],
            'nom_ca' : row['nom_ca'],
            'compromiso' : row['TOTAL COMPROMISO'],
            'entrega' : row['TOTAL ENTREGAS'],
            'entrega_porcen' : row['%ENTREGA']
            }
            list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
        # totales de los datos del grupo
        grupo_totales = {
            'compromiso' : grupo['TOTAL COMPROMISO'].sum(),
            'entregas' : grupo['TOTAL ENTREGAS'].sum(),
            'entrega_porcen' : grupo['TOTAL ENTREGAS'].sum() / grupo['TOTAL COMPROMISO'].sum() * 100
        }
        obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
        
        # datos generales de avance de cosecha
        avance = xlsx_avance[xlsx_avance['COD COS'] ==cod]
        avance_totales = {
            'fecha_img' : fecha_img,
            'fecha_informe' : fecha_inf,
            'compromiso' : float(avance['COMPROMISO'].iloc[0]),
            'entrega' : float(avance['ENTREGAS'].iloc[0]),
            'entrega_porcen' : float(avance['%ENTREGA'].iloc[0]) * 100,
            'ha_canha' : float(avance['COSECHADO'].iloc[0] + avance['SIN COSECHAR'].iloc[0]),
            'ha_cosecha' : float(avance['COSECHADO'].iloc[0]),
            'cosecha_porcen' : float(avance['%COSECHA'].iloc[0]) * 100,
            'tch' : float(avance['TCH'].iloc[0])
        }
        # datos generales a objeto
        obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
        
        # generar reporte
        doc = DocxTemplate(RUTA_COMPLETA + '/templates/tpl_rpt_avance_posible_desvio.docx')
        context = {
            'insp' : list_obj_caneros,
            'grupo_totales' : obj_grupo_totales,
            'avance' : obj_avance_totales
        }
        doc.render(context)

        #REPROTE POSIBLE COMPRA DE CAÑA
        doc.save(RUTA_COMPLETA + '/informes/' + str(avance['COD CA'].iloc[0]) + '_RPCC_' + fecha_form + '_' + avance['NOMBRE CAÑERO'].iloc[0] + '.docx')
        print('Se generó reporte de grupo:', cod, ' - ', avance['NOMBRE CAÑERO'].iloc[0])

In [10]:
crear_reporte(cod_grupos)

Se generó reporte de grupo: 166  -  PERALES GUERRERO IVAR
